![](../img/330-banner.png)

Lecture 6: `sklearn` `ColumnTransformer` and Text Features
------------
UBC 2022-23 W2

Instructor: Amir Abdi
 - Office Hours: Mondays 5-6 (or 5-7)

<br><br><br>

iclicker link: https://join.iclicker.com/EMMJ   
<img src="img_aa/iclicker_qr_code.png" height="300" width="300"> 


## Announcements, and LO

### Announcements

- Homework 3 is due Feb 1, 11:59pm
- We're working on Homework 2 grading. The grades will be released later this week.  

### Learning outcomes 

From this lecture, you will be able to 

- use `ColumnTransformer` to build all our transformations together into one object and use it with `sklearn` pipelines;  
- define `ColumnTransformer` where transformers contain more than one steps;
- explain `handle_unknown="ignore"` hyperparameter of `scikit-learn`'s `OneHotEncoder`;
- explain `drop="if_binary"` argument of `OneHotEncoder`;
- **identify when it's appropriate to apply ordinal encoding vs one-hot encoding;**
- **explain strategies to deal with categorical variables with too many categories;**

Text Data:
- explain why **text** data needs a different treatment than categorical variables;
- use `scikit-learn`'s `CountVectorizer` to encode text data;
- explain different hyperparameters of `CountVectorizer`.
- incorporate text features in a machine learning pipeline

## Legends

    
| <img src="https://upload.wikimedia.org/wikipedia/commons/f/f8/This_is_the_photo_of_Arthur_Samuel.jpg" width="100"> | <img src="http://www.cs.cmu.edu/~tom/TomHead2-6-22-22.jpg" width="100">  | <img src="https://upload.wikimedia.org/wikipedia/commons/4/49/John_McCarthy_Stanford.jpg" width="100"> | <img src="https://datascience.columbia.edu/wp-content/uploads/2020/08/Vapnik_web.png" width="100"> |
| :-----------: | :-----------: | :-----------: | :-----------: |
| Arthur Samuel       | Tom Mitchell       |John McCarthy|  Vladimir N. Vapnik |
| (1901-1990)    | 1951 - Now       |  1927 – 2011 | 1936 - Now |
| First computer learning program | 1997 ML Texbook, CMU Prof | Co-coined term AI, Lisp,<br> Time-sharing, Garbage collection | SVM



<img src="https://upload.wikimedia.org/wikipedia/commons/a/a1/Alan_Turing_Aged_16.jpg" width="300">

**Alan Turing**  
**1912 – 1954 (42 years)**

- Known for: 
  - Turing Test (1950):  test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human.
  - Turing Machine: The abstract machine, executing symbols on an infitnite stripe of tape, and capable of implementing any computer program.
  - and many more that is beyond **my limited scope of knowledge**...
- Turing was prosecuted in 1952 for homosexual acts. Turing did not deny the charges.
  - An official public apology was made on behalf of the British government for "the appalling way Turing was treated". 


Other places where you might hear his name:
- Turing Award (Noble Prize for Computing)
- Turing Completeness
- Alan Turing law: Refer to a 2017 law in the United Kingdom that retroactively pardoned men cautioned or convicted under historical legislation that outlawed homosexual acts.


> Turing “took his time finding the right words,” and BBC radio producer had called Turing a very difficult person to interview for that reason.

<br><br><br><br>

# sklearn's [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

**Imports**

In [1]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML

sys.path.append("../code/.")
pd.set_option("display.max_colwidth", 200)

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

- In most applications, some features are categorical, some are continuous, some are binary, and some are ordinal. 

- When we want to develop supervised machine learning pipelines on real-world datasets, very often we want to **apply different transformation on different columns**. 

- Enter `sklearn`'s `ColumnTransformer`!! 

- Let's look at a toy example: 

In [2]:
df = pd.read_csv("../data/quiz2-grade-toy-col-transformer.csv")
df

enjoy_course  ml_experience                   major class_attendance  \
0           yes              1        Computer Science        Excellent   
1           yes              1  Mechanical Engineering          Average   
2           yes              0             Mathematics             Poor   
3            no              0             Mathematics        Excellent   
4           yes              0              Psychology             Good   
5            no              1               Economics             Good   
6           yes              1        Computer Science        Excellent   
7            no              0  Mechanical Engineering             Poor   
8            no              0             Linguistics          Average   
9           yes              1             Mathematics          Average   
10          yes              0              Psychology             Good   
11          yes              1                 Physics          Average   
12          yes              1                 Physics        Excellent   
13          yes              0  Mechanical Engineering        Excellent   
14           no              0             Mathematics             Poor   
15           no              1        Computer Science             Good   
16          yes              0        Computer Science          Average   
17          yes              1               Economics          Average   
18           no              1                 Biology             Good   
19           no              0              Psychology             Poor   
20          yes              1             Linguistics        Excellent   

    university_years  lab1  lab2  lab3  lab4  quiz1   quiz2  
0                  3    92  93.0    84    91     92      A+  
1                  2    94  90.0    80    83     91  not A+  
2                  3    78  85.0    83    80     80  not A+  
3                  3    91   NaN    92    91     89      A+  
4                  4    77  83.0    90    92     85      A+  
5                  5    70  73.0    68    74     71  not A+  
6                  4    80  88.0    89    88     91      A+  
7                  3    95  93.0    69    79     75  not A+  
8                  2    97  90.0    94    82     80  not A+  
9                  4    95  82.0    94    94     85  not A+  
10                 3    98  86.0    95    95     78      A+  
11                 1    95  88.0    93    92     85      A+  
12                 2    98  96.0    96    99    100      A+  
13                 4    95  94.0    96    95    100      A+  
14                 3    95  90.0    93    95     70  not A+  
15                 3    92  85.0    67    94     92  not A+  
16                 5    75  91.0    93    86     85      A+  
17                 3    86  89.0    65    86     87  not A+  
18                 2    91   NaN    90    88     82  not A+  
19                 2    77  94.0    87    81     89  not A+  
20                 4    96  92.0    92    96     87      A+

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   enjoy_course      21 non-null     object 
 1   ml_experience     21 non-null     int64  
 2   major             21 non-null     object 
 3   class_attendance  21 non-null     object 
 4   university_years  21 non-null     int64  
 5   lab1              21 non-null     int64  
 6   lab2              19 non-null     float64
 7   lab3              21 non-null     int64  
 8   lab4              21 non-null     int64  
 9   quiz1             21 non-null     int64  
 10  quiz2             21 non-null     object 
dtypes: float64(1), int64(6), object(4)
memory usage: 1.9+ KB


## Transformations on the toy data

In [4]:
df.head()

enjoy_course  ml_experience                   major class_attendance  \
0          yes              1        Computer Science        Excellent   
1          yes              1  Mechanical Engineering          Average   
2          yes              0             Mathematics             Poor   
3           no              0             Mathematics        Excellent   
4          yes              0              Psychology             Good   

   university_years  lab1  lab2  lab3  lab4  quiz1   quiz2  
0                 3    92  93.0    84    91     92      A+  
1                 2    94  90.0    80    83     91  not A+  
2                 3    78  85.0    83    80     80  not A+  
3                 3    91   NaN    92    91     89      A+  
4                 4    77  83.0    90    92     85      A+

- Scaling on numeric features
- One-hot encoding on the categorical feature `major` and binary feature `enjoy_class`
- Ordinal encoding on the ordinal feature `class_attendance`
- Imputation on the `lab2` feature
- None on the `ml_experience` feature

In [5]:
X = df.drop(columns=["quiz2"])
y = df["quiz2"]
X.columns

Index(['enjoy_course', 'ml_experience', 'major', 'class_attendance',
       'university_years', 'lab1', 'lab2', 'lab3', 'lab4', 'quiz1'],
      dtype='object')

In [6]:
X.head()

enjoy_course  ml_experience                   major class_attendance  \
0          yes              1        Computer Science        Excellent   
1          yes              1  Mechanical Engineering          Average   
2          yes              0             Mathematics             Poor   
3           no              0             Mathematics        Excellent   
4          yes              0              Psychology             Good   

   university_years  lab1  lab2  lab3  lab4  quiz1  
0                 3    92  93.0    84    91     92  
1                 2    94  90.0    80    83     91  
2                 3    78  85.0    83    80     80  
3                 3    91   NaN    92    91     89  
4                 4    77  83.0    90    92     85

In [7]:
numeric_feats = ["university_years", "lab1", "lab3", "lab4", "quiz1"]  # apply scaling
categorical_feats = ["major"]  # apply one-hot encoding
passthrough_feats = ["ml_experience"]  # do not apply any transformation

# Not needed, as drop is the default behaviour
drop_feats = [
    "lab2",
    "class_attendance",
    "enjoy_course",
]  # do not include these features in modeling

For simplicity, let's only focus on scaling and one-hot encoding first. 

### `ColumnTransformer` Interface

- Each transformation is specified by a name, a transformer object, and the columns this transformer should be applied to. 

In [8]:
from sklearn.compose import ColumnTransformer

# -------- New Class ------------
ct = ColumnTransformer(
    [
        ("MyScaling", StandardScaler(), numeric_feats),
        ("MyOnehot", OneHotEncoder(sparse=False), categorical_feats),
        ("MyPassthrough", "passthrough", passthrough_feats),
        # ("MyDrop", "drop", drop_feats), # not neeeded, drop is the default behaviour
    ]
)
# -------------------------------
ct

ColumnTransformer(transformers=[('MyScaling', StandardScaler(),
                                 ['university_years', 'lab1', 'lab3', 'lab4',
                                  'quiz1']),
                                ('MyOnehot', OneHotEncoder(sparse=False),
                                 ['major']),
                                ('MyPassthrough', 'passthrough',
                                 ['ml_experience'])])

### `make_column_transformer` Interface

- Similar to `make_pipeline` syntax, there is convenient `make_column_transformer` syntax. 
- The syntax automatically names each step based on its class. 
- We'll be mostly using this syntax. 

In [9]:
from sklearn.compose import make_column_transformer

ct = make_column_transformer(    
    (StandardScaler(), numeric_feats),  # scaling on numeric features
    ("passthrough", passthrough_feats),  # no transformations on the binary features    
    (OneHotEncoder(), categorical_feats),  # OHE on categorical features
    # ("drop", drop_feats),   # not neeeded, drop is the default behaviour
)
ct

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['university_years', 'lab1', 'lab3', 'lab4',
                                  'quiz1']),
                                ('passthrough', 'passthrough',
                                 ['ml_experience']),
                                ('onehotencoder', OneHotEncoder(), ['major'])])

In [10]:
ct.fit(X)
transformed = ct.transform(X)

# if we had test data
# transform_X_test = ct.transform(X_test)

# Alternatively, you could have called:
# transformed = ct.fit_transform(X)

In [11]:
X.shape

(21, 10)

In [12]:
transformed.shape

(21, 14)

In [13]:
type(X)

pandas.core.frame.DataFrame

In [14]:
type(transformed)

numpy.ndarray

- When we `fit_transform`, each transformer is applied to the specified columns and the result of the transformations are **concatenates the results**. 
- A big advantage here is that we build all our transformations together into one object, and that way we're sure we do the same operations to all splits of the data.
- Otherwise we might, for example, do the OHE on both train and test but forget to scale the test data.

<br><br><br><br><br><br><br>
**[study at home]**

### Convert `numpy.ndarray`
<br><br>
Note that the returned object is not a dataframe. So there are no column names.  back to `DataFrame`

In [15]:
transformed

array([[-0.09345386,  0.3589134 , -0.21733442,  0.36269995,  0.84002795,
         1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-1.07471942,  0.59082668, -0.61420598, -0.85597188,  0.71219761,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ],
       [-0.09345386, -1.26447953, -0.31655231, -1.31297381, -0.69393613,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ],
       [-0.09345386,  0.24295676,  0.57640869,  0.36269995,  0.45653693,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.8878117 , -1.38043616,  0.37797291,  0.51503393, -0.05478443,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.  



- How can we view our transformed data as a dataframe? 
- We are adding more columns. 
- So the original columns won't directly map to the transformed data. 
- Let's create column names for the transformed data. 

In [16]:
column_names = (
    numeric_feats
    + passthrough_feats    
    + ct.named_transformers_["onehotencoder"].get_feature_names_out().tolist()
)
column_names

['university_years',
 'lab1',
 'lab3',
 'lab4',
 'quiz1',
 'ml_experience',
 'major_Biology',
 'major_Computer Science',
 'major_Economics',
 'major_Linguistics',
 'major_Mathematics',
 'major_Mechanical Engineering',
 'major_Physics',
 'major_Psychology']

In [17]:
ct.named_transformers_

{'standardscaler': StandardScaler(),
 'passthrough': 'passthrough',
 'onehotencoder': OneHotEncoder(),
 'remainder': 'drop'}

<br><br>
Note that the order of the columns in the transformed data depends upon the order of the features we pass to the `ColumnTransformer` and can be different than the order of the features in the original dataframe.  
<br><br>

In [18]:
pd.DataFrame(transformed, columns=column_names)

university_years      lab1      lab3      lab4     quiz1  ml_experience  \
0          -0.093454  0.358913 -0.217334  0.362700  0.840028            1.0   
1          -1.074719  0.590827 -0.614206 -0.855972  0.712198            1.0   
2          -0.093454 -1.264480 -0.316552 -1.312974 -0.693936            0.0   
3          -0.093454  0.242957  0.576409  0.362700  0.456537            0.0   
4           0.887812 -1.380436  0.377973  0.515034 -0.054784            0.0   
5           1.869077 -2.192133 -1.804821 -2.226978 -1.844409            1.0   
6           0.887812 -1.032566  0.278755 -0.094302  0.712198            1.0   
7          -0.093454  0.706783 -1.705603 -1.465308 -1.333088            0.0   
8          -1.074719  0.938697  0.774844 -1.008306 -0.693936            0.0   
9           0.887812  0.706783  0.774844  0.819702 -0.054784            1.0   
10         -0.093454  1.054653  0.874062  0.972036 -0.949597            0.0   
11         -2.055985  0.706783  0.675627  0.515034 -0.054784            1.0   
12         -1.074719  1.054653  0.973280  1.581372  1.862671            1.0   
13          0.887812  0.706783  0.973280  0.972036  1.862671            0.0   
14         -0.093454  0.706783  0.675627  0.972036 -1.972240            0.0   
15         -0.093454  0.358913 -1.904039  0.819702  0.840028            1.0   
16          1.869077 -1.612349  0.675627 -0.398970 -0.054784            0.0   
17         -0.093454 -0.336826 -2.102474 -0.398970  0.200876            1.0   
18         -1.074719  0.242957  0.377973 -0.094302 -0.438275            1.0   
19         -1.074719 -1.380436  0.080319 -1.160640  0.456537            0.0   
20          0.887812  0.822740  0.576409  1.124370  0.200876            1.0   

    major_Biology  major_Computer Science  major_Economics  major_Linguistics  \
0             0.0                     1.0              0.0                0.0   
1             0.0                     0.0              0.0                0.0   
2             0.0                     0.0              0.0                0.0   
3             0.0                     0.0              0.0                0.0   
4             0.0                     0.0              0.0                0.0   
5             0.0                     0.0              1.0                0.0   
6             0.0                     1.0              0.0                0.0   
7             0.0                     0.0              0.0                0.0   
8             0.0                     0.0              0.0                1.0   
9             0.0                     0.0              0.0                0.0   
10            0.0                     0.0              0.0                0.0   
11            0.0                     0.0              0.0                0.0   
12            0.0                     0.0              0.0                0.0   
13            0.0                     0.0              0.0                0.0   
14            0.0                     0.0              0.0                0.0   
15            0.0                     1.0              0.0                0.0   
16            0.0                     1.0              0.0                0.0   
17            0.0                     0.0              1.0                0.0   
18            1.0                     0.0              0.0                0.0   
19            0.0                     0.0              0.0                0.0   
20            0.0                     0.0              0.0                1.0   

    major_Mathematics  major_Mechanical Engineering  major_Physics  \
0                 0.0                           0.0            0.0   
1                 0.0                           1.0            0.0   
2                 1.0                           0.0            0.0   
3                 1.0                           0.0            0.0   
4                 0.0                           0.0            0.0   
5                 0.0                           0.0           

<br><br><br><br><br><br>
### Summary

<br>

![](../img/column-transformer.png)
<!-- <img src='./img/column-transformer.png' width="1500"> -->

[Adapted from here.](https://amueller.github.io/COMS4995-s20/slides/aml-04-preprocessing/#37)

## Training models with transformed data
- We can now pass the `ColumnTransformer` object as a step in a **pipeline**. 

In [19]:
# Same as before, just passing ColumnTransformer (ct) to pipeline
pipe = make_pipeline(ct, SVC())
pipe.fit(X, y)

# I don't care about results on train data; this is toy problem
pipe.predict(X)

array(['A+', 'not A+', 'not A+', 'A+', 'A+', 'not A+', 'A+', 'not A+',
       'not A+', 'A+', 'A+', 'A+', 'A+', 'A+', 'not A+', 'not A+', 'A+',
       'not A+', 'not A+', 'not A+', 'A+'], dtype=object)

In [20]:
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['university_years', 'lab1',
                                                   'lab3', 'lab4', 'quiz1']),
                                                 ('passthrough', 'passthrough',
                                                  ['ml_experience']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['major'])])),
                ('svc', SVC())])

<br><br>

## ❓❓ Questions for you 

### (iClicker) Exercise 6.1 

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

1. You could carry out cross-validation by passing a `ColumnTransformer` object directly to `cross_validate`. 
2. After applying column transformer, the order of the columns in the transformed data has to be the same as the order of the columns in the original data. 
3. After applying a column transformer, the transformed data is always going to be of different shape than the original data. 
4. When you call `fit_transform` on a `ColumnTransformer` object, you get a numpy ndarray. 

Answers True: 4

<br><br><br><br>

---------------
<br><br><br><br><br><br>
**[Study on your own - Random details on how to set the output type of ScikitLearn]**

### `sklearn` `set_config`

In [21]:
from sklearn import set_config

In [22]:
set_config(display="text")
ct

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['university_years', 'lab1', 'lab3', 'lab4',
                                  'quiz1']),
                                ('passthrough', 'passthrough',
                                 ['ml_experience']),
                                ('onehotencoder', OneHotEncoder(), ['major'])])

In [23]:
set_config(display="diagram")
ct

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['university_years', 'lab1', 'lab3', 'lab4',
                                  'quiz1']),
                                ('passthrough', 'passthrough',
                                 ['ml_experience']),
                                ('onehotencoder', OneHotEncoder(), ['major'])])

<br><br><br><br>

-----------------

### Multiple transformations in a transformer with pipeline

We can nest a pipeline inside a transformer

<br><br>

Recall that `lab2` has missing values. 


In [24]:
X.head(10)

enjoy_course  ml_experience                   major class_attendance  \
0          yes              1        Computer Science        Excellent   
1          yes              1  Mechanical Engineering          Average   
2          yes              0             Mathematics             Poor   
3           no              0             Mathematics        Excellent   
4          yes              0              Psychology             Good   
5           no              1               Economics             Good   
6          yes              1        Computer Science        Excellent   
7           no              0  Mechanical Engineering             Poor   
8           no              0             Linguistics          Average   
9          yes              1             Mathematics          Average   

   university_years  lab1  lab2  lab3  lab4  quiz1  
0                 3    92  93.0    84    91     92  
1                 2    94  90.0    80    83     91  
2                 3    78  85.0    83    80     80  
3                 3    91   NaN    92    91     89  
4                 4    77  83.0    90    92     85  
5                 5    70  73.0    68    74     71  
6                 4    80  88.0    89    88     91  
7                 3    95  93.0    69    79     75  
8                 2    97  90.0    94    82     80  
9                 4    95  82.0    94    94     85

- So we would like to apply more than one transformations on it: imputation and scaling.  
- We can treat `lab2` separately, but we can also include it into `numeric_feats` and apply both transformations on all numeric columns.

In [25]:
numeric_feats = [
    "university_years",
    "lab1",
    "lab2",
    "lab3",
    "lab4",
    "quiz1",
]  # apply scaling
categorical_feats = ["major"]  # apply one-hot encoding
passthrough_feats = ["ml_experience"]  # do not apply any transformation

# Not needed, default behaviour
drop_feats = ["class_attendance", "enjoy_course"]

<br><br><br>
**To apply more than one transformations we can define a pipeline inside a column transformer to chain different transformations.**
<br><br><br>

In [26]:
ct = make_column_transformer(
  # ---------- important -------------------
    (      
        make_pipeline(SimpleImputer(), StandardScaler()),
        numeric_feats,
    ),
    # -------------------------------------------
    ("passthrough", passthrough_feats),  # no transformations on the binary features    
    (OneHotEncoder(), categorical_feats),  # OHE on categorical features
)

In [27]:
ct

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['university_years', 'lab1', 'lab2', 'lab3',
                                  'lab4', 'quiz1']),
                                ('passthrough', 'passthrough',
                                 ['ml_experience']),
                                ('onehotencoder', OneHotEncoder(), ['major'])])

In [28]:
X_transformed = ct.fit_transform(X)

In [29]:
column_names = (
    numeric_feats
    + passthrough_feats    
    + ct.named_transformers_["onehotencoder"].get_feature_names_out().tolist()
)
column_names

['university_years',
 'lab1',
 'lab2',
 'lab3',
 'lab4',
 'quiz1',
 'ml_experience',
 'major_Biology',
 'major_Computer Science',
 'major_Economics',
 'major_Linguistics',
 'major_Mathematics',
 'major_Mechanical Engineering',
 'major_Physics',
 'major_Psychology']

In [30]:
pd.DataFrame(X_transformed, columns=column_names)

university_years      lab1      lab2      lab3      lab4     quiz1  \
0          -0.093454  0.358913  0.893260 -0.217334  0.362700  0.840028   
1          -1.074719  0.590827  0.294251 -0.614206 -0.855972  0.712198   
2          -0.093454 -1.264480 -0.704099 -0.316552 -1.312974 -0.693936   
3          -0.093454  0.242957  0.000000  0.576409  0.362700  0.456537   
4           0.887812 -1.380436 -1.103439  0.377973  0.515034 -0.054784   
5           1.869077 -2.192133 -3.100139 -1.804821 -2.226978 -1.844409   
6           0.887812 -1.032566 -0.105089  0.278755 -0.094302  0.712198   
7          -0.093454  0.706783  0.893260 -1.705603 -1.465308 -1.333088   
8          -1.074719  0.938697  0.294251  0.774844 -1.008306 -0.693936   
9           0.887812  0.706783 -1.303109  0.774844  0.819702 -0.054784   
10         -0.093454  1.054653 -0.504429  0.874062  0.972036 -0.949597   
11         -2.055985  0.706783 -0.105089  0.675627  0.515034 -0.054784   
12         -1.074719  1.054653  1.492270  0.973280  1.581372  1.862671   
13          0.887812  0.706783  1.092930  0.973280  0.972036  1.862671   
14         -0.093454  0.706783  0.294251  0.675627  0.972036 -1.972240   
15         -0.093454  0.358913 -0.704099 -1.904039  0.819702  0.840028   
16          1.869077 -1.612349  0.493921  0.675627 -0.398970 -0.054784   
17         -0.093454 -0.336826  0.094581 -2.102474 -0.398970  0.200876   
18         -1.074719  0.242957  0.000000  0.377973 -0.094302 -0.438275   
19         -1.074719 -1.380436  1.092930  0.080319 -1.160640  0.456537   
20          0.887812  0.822740  0.693590  0.576409  1.124370  0.200876   

    ml_experience  major_Biology  major_Computer Science  major_Economics  \
0             1.0            0.0                     1.0              0.0   
1             1.0            0.0                     0.0              0.0   
2             0.0            0.0                     0.0              0.0   
3             0.0            0.0                     0.0              0.0   
4             0.0            0.0                     0.0              0.0   
5             1.0            0.0                     0.0              1.0   
6             1.0            0.0                     1.0              0.0   
7             0.0            0.0                     0.0              0.0   
8             0.0            0.0                     0.0              0.0   
9             1.0            0.0                     0.0              0.0   
10            0.0            0.0                     0.0              0.0   
11            1.0            0.0                     0.0              0.0   
12            1.0            0.0                     0.0              0.0   
13            0.0            0.0                     0.0              0.0   
14            0.0            0.0                     0.0              0.0   
15            1.0            0.0                     1.0              0.0   
16            0.0            0.0                     1.0              0.0   
17            1.0            0.0                     0.0              1.0   
18            1.0            1.0                     0.0              0.0   
19            0.0            0.0                     0.0              0.0   
20            1.0            0.0                     0.0              0.0   

    major_Linguistics  major_Mathematics  major_Mechanical Engineering  \
0                 0.0                0.0                           0.0   
1                 0.0                0.0                           1.0   
2                 0.0                1.0                           0.0   
3                 0.0                1.0                           0.0   
4                 0.0                0.0                           0.0   
5                 0.0                0.0                           0.0   
6                 0.0                0.0                           0.0   
7                 0.0                0.0                           1.0   
8             

<br><br>

<br><br>

### Incorporating ordinal feature `class_attendance` 

- The `class_attendance` column is different than the `major` column in that there is some ordering of the values. 
    - Excellent > Good > Average > poor    

In [31]:
X.head()

enjoy_course  ml_experience                   major class_attendance  \
0          yes              1        Computer Science        Excellent   
1          yes              1  Mechanical Engineering          Average   
2          yes              0             Mathematics             Poor   
3           no              0             Mathematics        Excellent   
4          yes              0              Psychology             Good   

   university_years  lab1  lab2  lab3  lab4  quiz1  
0                 3    92  93.0    84    91     92  
1                 2    94  90.0    80    83     91  
2                 3    78  85.0    83    80     80  
3                 3    91   NaN    92    91     89  
4                 4    77  83.0    90    92     85

Let's try applying `OrdinalEncoder` on this column. 

In [32]:
X_toy = X[["class_attendance"]]
enc = OrdinalEncoder()
enc.fit(X_toy)
X_toy_ord = enc.transform(X_toy)
df = pd.DataFrame(
    data=X_toy_ord,
    columns=["class_attendance_enc"],
    index=X_toy.index,
)

In [33]:
pd.concat([X_toy, df], axis=1).head(10)

class_attendance  class_attendance_enc
0        Excellent                   1.0
1          Average                   0.0
2             Poor                   3.0
3        Excellent                   1.0
4             Good                   2.0
5             Good                   2.0
6        Excellent                   1.0
7             Poor                   3.0
8          Average                   0.0
9          Average                   0.0

- What's the problem here? 
    - The encoder doesn't know the order. 
- We can examine unique categories manually, order them based on our intuitions, and then provide this human knowledge to the transformer. 

What are the unique categories of `class_attendance`? 

In [34]:
X_toy["class_attendance"].unique()

array(['Excellent', 'Average', 'Poor', 'Good'], dtype=object)

**Let's order them manually.**

In [35]:
class_attendance_levels = ["Poor", "Average", "Good", "Excellent"]

Let's make sure that we have included all categories in our manual ordering.  

In [36]:
assert set(class_attendance_levels) == set(X_toy["class_attendance"].unique())

In [37]:
oe = OrdinalEncoder(categories=[class_attendance_levels], dtype=int)
oe.fit(X_toy[["class_attendance"]])
ca_transformed = oe.transform(X_toy[["class_attendance"]])
df = pd.DataFrame(
    data=ca_transformed, columns=["class_attendance_enc"], index=X_toy.index
)
print(oe.categories_)
pd.concat([X_toy, df], axis=1).head(10)

[array(['Poor', 'Average', 'Good', 'Excellent'], dtype=object)]


class_attendance  class_attendance_enc
0        Excellent                     3
1          Average                     1
2             Poor                     0
3        Excellent                     3
4             Good                     2
5             Good                     2
6        Excellent                     3
7             Poor                     0
8          Average                     1
9          Average                     1

The encoded categories are looking better now! 

#### More than one ordinal columns?

- We can pass the manually ordered categories when we create an `OrdinalEncoder` object as a list of lists. 
- If you have more than one ordinal columns
    - manually create a list of ordered categories for each column
    - pass a list of lists to `OrdinalEncoder`, where each inner list corresponds to manually created list of ordered categories for a corresponding ordinal column. 
    

Now let's incorporate ordinal encoding of `class_attendance` in our column transformer. 

In [38]:
X

enjoy_course  ml_experience                   major class_attendance  \
0           yes              1        Computer Science        Excellent   
1           yes              1  Mechanical Engineering          Average   
2           yes              0             Mathematics             Poor   
3            no              0             Mathematics        Excellent   
4           yes              0              Psychology             Good   
5            no              1               Economics             Good   
6           yes              1        Computer Science        Excellent   
7            no              0  Mechanical Engineering             Poor   
8            no              0             Linguistics          Average   
9           yes              1             Mathematics          Average   
10          yes              0              Psychology             Good   
11          yes              1                 Physics          Average   
12          yes              1                 Physics        Excellent   
13          yes              0  Mechanical Engineering        Excellent   
14           no              0             Mathematics             Poor   
15           no              1        Computer Science             Good   
16          yes              0        Computer Science          Average   
17          yes              1               Economics          Average   
18           no              1                 Biology             Good   
19           no              0              Psychology             Poor   
20          yes              1             Linguistics        Excellent   

    university_years  lab1  lab2  lab3  lab4  quiz1  
0                  3    92  93.0    84    91     92  
1                  2    94  90.0    80    83     91  
2                  3    78  85.0    83    80     80  
3                  3    91   NaN    92    91     89  
4                  4    77  83.0    90    92     85  
5                  5    70  73.0    68    74     71  
6                  4    80  88.0    89    88     91  
7                  3    95  93.0    69    79     75  
8                  2    97  90.0    94    82     80  
9                  4    95  82.0    94    94     85  
10                 3    98  86.0    95    95     78  
11                 1    95  88.0    93    92     85  
12                 2    98  96.0    96    99    100  
13                 4    95  94.0    96    95    100  
14                 3    95  90.0    93    95     70  
15                 3    92  85.0    67    94     92  
16                 5    75  91.0    93    86     85  
17                 3    86  89.0    65    86     87  
18                 2    91   NaN    90    88     82  
19                 2    77  94.0    87    81     89  
20                 4    96  92.0    92    96     87

In [39]:
numeric_feats = [
    "university_years",
    "lab1",
    "lab2",
    "lab3",
    "lab4",
    "quiz1",
]  # apply scaling
categorical_feats = ["major"]  # apply one-hot encoding
ordinal_feats = ["class_attendance"]  # apply ordinal encoding
passthrough_feats = ["ml_experience"]  # do not apply any transformation
drop_feats = ["enjoy_course"]  # do not include these features

In [40]:
ct = make_column_transformer(
    (
        make_pipeline(SimpleImputer(), StandardScaler()),
        numeric_feats,
    ),
    (
        OrdinalEncoder(categories=[class_attendance_levels], dtype=int),
        ordinal_feats,
    ),  # Ordinal encoding on ordinal features
    ("passthrough", passthrough_feats),  # no transformations on the binary features
    (OneHotEncoder(), categorical_feats),  # OHE on categorical features    
)

In [41]:
ct

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['university_years', 'lab1', 'lab2', 'lab3',
                                  'lab4', 'quiz1']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['Poor', 'Average',
                                                             'Good',
                                                             'Excellent']],
                                                dtype=<class 'int'>),
                                 ['class_attendance']),
                                ('passthrough', 'passthrough',
                                 ['ml_experience']),
                                ('onehotencoder', OneHotEncoder(), ['major'])])

In [42]:
X_transformed = ct.fit_transform(X)

In [43]:
column_names = (
    numeric_feats
    + ordinal_feats
    + passthrough_feats    
    + ct.named_transformers_["onehotencoder"].get_feature_names_out().tolist()
)
column_names

['university_years',
 'lab1',
 'lab2',
 'lab3',
 'lab4',
 'quiz1',
 'class_attendance',
 'ml_experience',
 'major_Biology',
 'major_Computer Science',
 'major_Economics',
 'major_Linguistics',
 'major_Mathematics',
 'major_Mechanical Engineering',
 'major_Physics',
 'major_Psychology']

In [44]:
pd.DataFrame(X_transformed, columns=column_names)

university_years      lab1      lab2      lab3      lab4     quiz1  \
0          -0.093454  0.358913  0.893260 -0.217334  0.362700  0.840028   
1          -1.074719  0.590827  0.294251 -0.614206 -0.855972  0.712198   
2          -0.093454 -1.264480 -0.704099 -0.316552 -1.312974 -0.693936   
3          -0.093454  0.242957  0.000000  0.576409  0.362700  0.456537   
4           0.887812 -1.380436 -1.103439  0.377973  0.515034 -0.054784   
5           1.869077 -2.192133 -3.100139 -1.804821 -2.226978 -1.844409   
6           0.887812 -1.032566 -0.105089  0.278755 -0.094302  0.712198   
7          -0.093454  0.706783  0.893260 -1.705603 -1.465308 -1.333088   
8          -1.074719  0.938697  0.294251  0.774844 -1.008306 -0.693936   
9           0.887812  0.706783 -1.303109  0.774844  0.819702 -0.054784   
10         -0.093454  1.054653 -0.504429  0.874062  0.972036 -0.949597   
11         -2.055985  0.706783 -0.105089  0.675627  0.515034 -0.054784   
12         -1.074719  1.054653  1.492270  0.973280  1.581372  1.862671   
13          0.887812  0.706783  1.092930  0.973280  0.972036  1.862671   
14         -0.093454  0.706783  0.294251  0.675627  0.972036 -1.972240   
15         -0.093454  0.358913 -0.704099 -1.904039  0.819702  0.840028   
16          1.869077 -1.612349  0.493921  0.675627 -0.398970 -0.054784   
17         -0.093454 -0.336826  0.094581 -2.102474 -0.398970  0.200876   
18         -1.074719  0.242957  0.000000  0.377973 -0.094302 -0.438275   
19         -1.074719 -1.380436  1.092930  0.080319 -1.160640  0.456537   
20          0.887812  0.822740  0.693590  0.576409  1.124370  0.200876   

    class_attendance  ml_experience  major_Biology  major_Computer Science  \
0                3.0            1.0            0.0                     1.0   
1                1.0            1.0            0.0                     0.0   
2                0.0            0.0            0.0                     0.0   
3                3.0            0.0            0.0                     0.0   
4                2.0            0.0            0.0                     0.0   
5                2.0            1.0            0.0                     0.0   
6                3.0            1.0            0.0                     1.0   
7                0.0            0.0            0.0                     0.0   
8                1.0            0.0            0.0                     0.0   
9                1.0            1.0            0.0                     0.0   
10               2.0            0.0            0.0                     0.0   
11               1.0            1.0            0.0                     0.0   
12               3.0            1.0            0.0                     0.0   
13               3.0            0.0            0.0                     0.0   
14               0.0            0.0            0.0                     0.0   
15               2.0            1.0            0.0                     1.0   
16               1.0            0.0            0.0                     1.0   
17               1.0            1.0            0.0                     0.0   
18               2.0            1.0            1.0                     0.0   
19               0.0            0.0            0.0                     0.0   
20               3.0            1.0            0.0                     0.0   

    major_Economics  major_Linguistics  major_Mathematics  \
0               0.0                0.0                0.0   
1               0.0                0.0                0.0   
2               0.0                0.0                1.0   
3               0.0                0.0                1.0   
4               0.0                0.0                0.0   
5               1.0                0.0                0.0   
6               0.0                0.0                0.0   
7               0.0                0.0                0.0   
8               0.0                1.0                0.0   
9               0.0                0.0          

<br><br>

### Dealing with unknown categories

Let's create a pipeline with the column transformer and pass it to `cross_validate`. 

In [45]:
pipe = make_pipeline(ct, SVC())
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['university_years', 'lab1',
                                                   'lab2', 'lab3', 'lab4',
                                                   'quiz1']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Poor',
                                                                              'Average',
                                                                              'Good',
                                                                              'Excellent']],
                                                                 dtype=<class 'int'>),
                                                  ['class_attendance']),
                                                 ('passthrough', 'passthrough',
                                                  ['ml_experience']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['major'])])),
                ('svc', SVC())])

In [46]:
# This will fails
# scores = cross_validate(pipe, X, y, return_train_score=True)

- What's going on here??
- Let's look at the error message:
`ValueError: Found unknown categories ['Biology'] in column 0 during transform
`

In [47]:
X["major"].value_counts()

Computer Science          4
Mathematics               4
Mechanical Engineering    3
Psychology                3
Economics                 2
Linguistics               2
Physics                   2
Biology                   1
Name: major, dtype: int64

- **There is only one instance of Biology.**
- During **cross-validation**, this is getting put into the validation split.
- By default, `OneHotEncoder` throws an error because you might want to know about this.

Simplest fix:
- Pass `handle_unknown="ignore"` argument to `OneHotEncoder`
- It creates a row with all zeros. 

In [48]:
ct = make_column_transformer(
    (
        make_pipeline(SimpleImputer(), StandardScaler()),
        numeric_feats,
    ),
    (
        OrdinalEncoder(categories=[class_attendance_levels], dtype=int),
        ordinal_feats,
    ),
    ("passthrough", passthrough_feats),
    (
        OneHotEncoder(handle_unknown="ignore"), # --> new code
        categorical_feats,
    ),
)

In [49]:
ct

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['university_years', 'lab1', 'lab2', 'lab3',
                                  'lab4', 'quiz1']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['Poor', 'Average',
                                                             'Good',
                                                             'Excellent']],
                                                dtype=<class 'int'>),
                                 ['class_attendance']),
                                ('passthrough', 'passthrough',
                                 ['ml_experience']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['major'])])

In [50]:
pipe = make_pipeline(ct, SVC())
scores = cross_validate(pipe, X, y, cv=5, return_train_score=True)
scores['test_score'].mean()

0.8

- With this approach, all unknown categories will be represented with all zeros and cross-validation is running OK now. 

Ask yourself the following questions when you work with categorical variables   
- Do you want this behaviour? 
- Are you expecting to get many unknown categories? Do you want to be able to distinguish between them?

**Learning about all possible categories of a given feature doesn't break the Golden Rule** because:
- The Train and Test data was supposed to originate from the **same distribution**.
- If it's some fix number of categories. For example, if it's something like provinces in Canada or majors taught at UBC. We know the categories in advance and this is one of the cases where it might be OK to violate the golden rule and get a list of all possible values for the categorical variable. 

<br><br>

In [51]:
X["enjoy_course"].head()

0    yes
1    yes
2    yes
3     no
4    yes
Name: enjoy_course, dtype: object

In [52]:
ohe_enc = OneHotEncoder(drop="if_binary", dtype=int, sparse=False)
ohe_enc.fit(X[["enjoy_course"]])
transformed = ohe_enc.transform(X[["enjoy_course"]])
df = pd.DataFrame(data=transformed, columns=["enjoy_course_enc"], index=X.index)
pd.concat([X[["enjoy_course"]], df], axis=1).head(10)

enjoy_course  enjoy_course_enc
0          yes                 1
1          yes                 1
2          yes                 1
3           no                 0
4          yes                 1
5           no                 0
6          yes                 1
7           no                 0
8           no                 0
9          yes                 1

In [53]:
numeric_feats = [
    "university_years",
    "lab1",
    "lab2",
    "lab3",
    "lab4",
    "quiz1",
]  # apply scaling
categorical_feats = ["major"]  # apply one-hot encoding
ordinal_feats = ["class_attendance"]  # apply ordinal encoding
binary_feats = ["enjoy_course"]  # apply one-hot encoding with drop="if_binary"
passthrough_feats = ["ml_experience"]  # do not apply any transformation
drop_feats = []

In [54]:
ct = make_column_transformer(
    (
        make_pipeline(SimpleImputer(), StandardScaler()),
        numeric_feats,
    ),
    (
        OrdinalEncoder(categories=[class_attendance_levels], dtype=int),
        ordinal_feats,
    ),
    (
        OneHotEncoder(drop="if_binary", dtype=int),  # --> new code
        binary_feats,
    ),  # OHE on categorical features
    ("passthrough", passthrough_feats),
    (
        OneHotEncoder(handle_unknown="ignore"),
        categorical_feats,
    )
)

In [55]:
ct

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['university_years', 'lab1', 'lab2', 'lab3',
                                  'lab4', 'quiz1']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['Poor', 'Average',
                                                             'Good',
                                                             'Excellent']],
                                                dtype=<class 'int'>),
                                 ['class_attendance']),
                                ('onehotencoder-1',
                                 OneHotEncoder(drop='if_binary',
                                               dtype=<class 'int'>),
                                 ['enjoy_course']),
                                ('passthrough', 'passthrough',
                                 ['ml_experience']),
                                ('onehotencoder-2',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['major'])])

In [56]:
pipe = make_pipeline(ct, SVC())
scores = cross_validate(pipe, X, y, cv=5, return_train_score=True)

## Break (5 min)

![](../img/eva-coffee.png)


<br><br><br><br>

---------
**[Study the section at home]**
# End2end example

In [57]:
housing_df = pd.read_csv("../data/housing.csv")
train_df, test_df = train_test_split(housing_df, test_size=0.1, random_state=123)

train_df.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
6051     -117.75     34.04                22.0       2948.0           636.0   
20113    -119.57     37.94                17.0        346.0           130.0   
14289    -117.13     32.74                46.0       3355.0           768.0   
13665    -117.31     34.02                18.0       1634.0           274.0   
14471    -117.23     32.88                18.0       5566.0          1465.0   

       population  households  median_income  median_house_value  \
6051       2600.0       602.0         3.1250            113600.0   
20113        51.0        20.0         3.4861            137500.0   
14289      1457.0       708.0         2.6604            170100.0   
13665       899.0       285.0         5.2139            129300.0   
14471      6303.0      1458.0         1.8580            205000.0   

      ocean_proximity  
6051           INLAND  
20113          INLAND  
14289      NEAR OCEAN  
13665          INLAND  
14471      NEAR OCEAN

Some column values are mean/median but some are not. 

Let's add some new features to the dataset which could help predicting the target: `median_house_value`. 

In [58]:
train_df = train_df.assign(
    rooms_per_household=train_df["total_rooms"] / train_df["households"]
)
test_df = test_df.assign(
    rooms_per_household=test_df["total_rooms"] / test_df["households"]
)

train_df = train_df.assign(
    bedrooms_per_household=train_df["total_bedrooms"] / train_df["households"]
)
test_df = test_df.assign(
    bedrooms_per_household=test_df["total_bedrooms"] / test_df["households"]
)

train_df = train_df.assign(
    population_per_household=train_df["population"] / train_df["households"]
)
test_df = test_df.assign(
    population_per_household=test_df["population"] / test_df["households"]
)

In [59]:
train_df.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
6051     -117.75     34.04                22.0       2948.0           636.0   
20113    -119.57     37.94                17.0        346.0           130.0   
14289    -117.13     32.74                46.0       3355.0           768.0   
13665    -117.31     34.02                18.0       1634.0           274.0   
14471    -117.23     32.88                18.0       5566.0          1465.0   

       population  households  median_income  median_house_value  \
6051       2600.0       602.0         3.1250            113600.0   
20113        51.0        20.0         3.4861            137500.0   
14289      1457.0       708.0         2.6604            170100.0   
13665       899.0       285.0         5.2139            129300.0   
14471      6303.0      1458.0         1.8580            205000.0   

      ocean_proximity  rooms_per_household  bedrooms_per_household  \
6051           INLAND             4.897010                1.056478   
20113          INLAND            17.300000                6.500000   
14289      NEAR OCEAN             4.738701                1.084746   
13665          INLAND             5.733333                0.961404   
14471      NEAR OCEAN             3.817558                1.004801   

       population_per_household  
6051                   4.318937  
20113                  2.550000  
14289                  2.057910  
13665                  3.154386  
14471                  4.323045

In [60]:
# Let's keep both numeric and categorical columns in the data.
X_train = train_df.drop(columns=["median_house_value", "total_rooms", "total_bedrooms", "population"])
y_train = train_df["median_house_value"]

X_test = test_df.drop(columns=["median_house_value", "total_rooms", "total_bedrooms", "population"])
y_test = test_df["median_house_value"]

In [61]:
from sklearn.compose import ColumnTransformer, make_column_transformer

In [62]:
X_train.head(10)

longitude  latitude  housing_median_age  households  median_income  \
6051     -117.75     34.04                22.0       602.0         3.1250   
20113    -119.57     37.94                17.0        20.0         3.4861   
14289    -117.13     32.74                46.0       708.0         2.6604   
13665    -117.31     34.02                18.0       285.0         5.2139   
14471    -117.23     32.88                18.0      1458.0         1.8580   
9730     -121.74     36.79                16.0       611.0         4.3814   
14690    -117.09     32.80                36.0       360.0         4.7188   
7938     -118.11     33.86                33.0       393.0         5.3889   
18365    -122.12     37.28                21.0        56.0         5.8691   
10931    -117.91     33.74                25.0       922.0         2.9926   

      ocean_proximity  rooms_per_household  bedrooms_per_household  \
6051           INLAND             4.897010                1.056478   
20113          INLAND            17.300000                6.500000   
14289      NEAR OCEAN             4.738701                1.084746   
13665          INLAND             5.733333                0.961404   
14471      NEAR OCEAN             3.817558                1.004801   
9730        <1H OCEAN             6.286416                1.014730   
14690      NEAR OCEAN             6.008333                1.019444   
7938        <1H OCEAN             6.078880                1.043257   
18365       <1H OCEAN             6.232143                1.142857   
10931       <1H OCEAN             4.634490                1.046638   

       population_per_household  
6051                   4.318937  
20113                  2.550000  
14289                  2.057910  
13665                  3.154386  
14471                  4.323045  
9730                   2.944354  
14690                  2.541667  
7938                   3.127226  
18365                  2.660714  
10931                  3.195228

In [63]:
X_train.columns

Index(['longitude', 'latitude', 'housing_median_age', 'households',
       'median_income', 'ocean_proximity', 'rooms_per_household',
       'bedrooms_per_household', 'population_per_household'],
      dtype='object')

In [64]:
# Identify the categorical and numeric columns
numeric_features = [
    "longitude",
    "latitude",
    "housing_median_age",
    "households",
    "median_income",
    "rooms_per_household",
    "bedrooms_per_household",
    "population_per_household",
]

categorical_features = ["ocean_proximity"]
target = "median_income"

- Let's create a `ColumnTransformer` for our dataset. 

In [65]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18576 entries, 6051 to 19966
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   longitude                 18576 non-null  float64
 1   latitude                  18576 non-null  float64
 2   housing_median_age        18576 non-null  float64
 3   households                18576 non-null  float64
 4   median_income             18576 non-null  float64
 5   ocean_proximity           18576 non-null  object 
 6   rooms_per_household       18576 non-null  float64
 7   bedrooms_per_household    18391 non-null  float64
 8   population_per_household  18576 non-null  float64
dtypes: float64(8), object(1)
memory usage: 1.4+ MB


In [66]:
X_train["ocean_proximity"].value_counts()

<1H OCEAN     8221
INLAND        5915
NEAR OCEAN    2389
NEAR BAY      2046
ISLAND           5
Name: ocean_proximity, dtype: int64

In [67]:
numeric_transformer = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
    (categorical_transformer, categorical_features),
)

In [68]:
preprocessor

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['longitude', 'latitude', 'housing_median_age',
                                  'households', 'median_income',
                                  'rooms_per_household',
                                  'bedrooms_per_household',
                                  'population_per_household']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['ocean_proximity'])])

In [69]:
X_train_pp = preprocessor.fit_transform(X_train)

- When we `fit` the preprocessor, it calls `fit` on _all_ the transformers
- When we `transform` the preprocessor, it calls `transform` on _all_ the transformers. 

We can get the new names of the columns that were generated by the one-hot encoding:

In [70]:
preprocessor

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['longitude', 'latitude', 'housing_median_age',
                                  'households', 'median_income',
                                  'rooms_per_household',
                                  'bedrooms_per_household',
                                  'population_per_household']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['ocean_proximity'])])

In [71]:
preprocessor.named_transformers_["onehotencoder"].get_feature_names_out(
    categorical_features
)

array(['ocean_proximity_<1H OCEAN', 'ocean_proximity_INLAND',
       'ocean_proximity_ISLAND', 'ocean_proximity_NEAR BAY',
       'ocean_proximity_NEAR OCEAN'], dtype=object)

Combining this with the numeric feature names gives us all the column names:

In [72]:
column_names = numeric_features + list(
    preprocessor.named_transformers_["onehotencoder"].get_feature_names_out(
        categorical_features
    )
)
column_names

['longitude',
 'latitude',
 'housing_median_age',
 'households',
 'median_income',
 'rooms_per_household',
 'bedrooms_per_household',
 'population_per_household',
 'ocean_proximity_<1H OCEAN',
 'ocean_proximity_INLAND',
 'ocean_proximity_ISLAND',
 'ocean_proximity_NEAR BAY',
 'ocean_proximity_NEAR OCEAN']

Let's visualize the preprocessed training data as a dataframe. 

In [73]:
pd.DataFrame(X_train_pp, columns=column_names)

longitude  latitude  housing_median_age  households  median_income  \
0       0.908140 -0.743917           -0.526078    0.266135      -0.389736   
1      -0.002057  1.083123           -0.923283   -1.253312      -0.198924   
2       1.218207 -1.352930            1.380504    0.542873      -0.635239   
3       1.128188 -0.753286           -0.843842   -0.561467       0.714077   
4       1.168196 -1.287344           -0.843842    2.500924      -1.059242   
...          ...       ...                 ...         ...            ...   
18571   0.733102 -0.804818            0.586095   -0.966131      -0.118182   
18572   1.163195 -1.057793           -1.161606    0.728235       0.357500   
18573  -1.097293  0.797355           -1.876574    0.514155       0.934269   
18574  -1.437367  1.008167            1.221622   -0.454427       0.006578   
18575   0.242996  0.272667           -0.684960   -0.396991      -0.711754   

       rooms_per_household  bedrooms_per_household  population_per_household  \
0                -0.210591               -0.083813                  0.126398   
1                 4.726412               11.166631                 -0.050132   
2                -0.273606               -0.025391                 -0.099240   
3                 0.122307               -0.280310                  0.010183   
4                -0.640266               -0.190617                  0.126808   
...                    ...                     ...                       ...   
18571             0.063110               -0.099558                  0.071541   
18572             0.235096               -0.163397                  0.007458   
18573             0.211892               -0.135305                  0.044029   
18574            -0.273382               -0.149822                 -0.132875   
18575             0.025998                0.042957                  0.051269   

       ocean_proximity_<1H OCEAN  ocean_proximity_INLAND  \
0                            0.0                     1.0   
1                            0.0                     1.0   
2                            0.0                     0.0   
3                            0.0                     1.0   
4                            0.0                     0.0   
...                          ...                     ...   
18571                        1.0                     0.0   
18572                        1.0                     0.0   
18573                        1.0                     0.0   
18574                        0.0                     0.0   
18575                        0.0                     1.0   

       ocean_proximity_ISLAND  ocean_proximity_NEAR BAY  \
0                         0.0                       0.0   
1                         0.0                       0.0   
2                         0.0                       0.0   
3                         0.0                       0.0   
4                         0.0                       0.0   
...                       ...                       ...   
18571                     0.0                       0.0   
18572                     0.0                       0.0   
18573                     0.0                       0.0   
18574                     0.0                       1.0   
18575                     0.0                       0.0   

       ocean_proximity_NEAR OCEAN  
0                             0.0  
1                             0.0  
2                             1.0  
3                             0.0  
4                             1.0  
...                           ...  
18571                         0.0  
18572                         0.0  
18573                         0.0  
18574                         0.0  
18575                         0.0  

[18576 rows x 13 columns]

In [74]:
from utils import mean_std_cross_val_scores
results_dict = {}
dummy = DummyRegressor()
results_dict["dummy"] = mean_std_cross_val_scores(
    dummy, X_train, y_train, return_train_score=True
)
pd.DataFrame(results_dict).T

fit_time         score_time          test_score  \
dummy  0.002 (+/- 0.002)  0.000 (+/- 0.000)  -0.001 (+/- 0.001)   

             train_score  
dummy  0.000 (+/- 0.000)

In [75]:
from sklearn.neighbors import KNeighborsRegressor
knn_pipe = make_pipeline(preprocessor, KNeighborsRegressor())

In [76]:
knn_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'households',
                                                   'median_income',
                                                   'rooms_per_household',
                                                   'bedrooms_per_household',
                                                   'population_per_household']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['ocean_proximity'])])),
                ('kneighborsregressor', KNeighborsRegressor())])

In [77]:
results_dict["imp + scaling + ohe + KNN"] = mean_std_cross_val_scores(
    knn_pipe, X_train, y_train, return_train_score=True
)

In [78]:
pd.DataFrame(results_dict).T

fit_time         score_time  \
dummy                      0.002 (+/- 0.002)  0.000 (+/- 0.000)   
imp + scaling + ohe + KNN  0.046 (+/- 0.008)  0.434 (+/- 0.062)   

                                   test_score        train_score  
dummy                      -0.001 (+/- 0.001)  0.000 (+/- 0.000)  
imp + scaling + ohe + KNN   0.706 (+/- 0.015)  0.806 (+/- 0.015)

In [79]:
from sklearn.svm import SVR

svr_pipe = make_pipeline(preprocessor, SVR())
results_dict["imp + scaling + ohe + SVR (default)"] = mean_std_cross_val_scores(
    svr_pipe, X_train, y_train, return_train_score=True
)

In [80]:
pd.DataFrame(results_dict).T

fit_time         score_time  \
dummy                                 0.002 (+/- 0.002)  0.000 (+/- 0.000)   
imp + scaling + ohe + KNN             0.046 (+/- 0.008)  0.434 (+/- 0.062)   
imp + scaling + ohe + SVR (default)  15.337 (+/- 0.311)  7.757 (+/- 0.112)   

                                             test_score         train_score  
dummy                                -0.001 (+/- 0.001)   0.000 (+/- 0.000)  
imp + scaling + ohe + KNN             0.706 (+/- 0.015)   0.806 (+/- 0.015)  
imp + scaling + ohe + SVR (default)  -0.049 (+/- 0.012)  -0.048 (+/- 0.001)

The results with `scikit-learn`'s default SVR hyperparameters are pretty bad. 

In [81]:
svr_C_pipe = make_pipeline(preprocessor, SVR(C=10000))
results_dict["imp + scaling + ohe + SVR (C=10000)"] = mean_std_cross_val_scores(
    svr_C_pipe, X_train, y_train, return_train_score=True
)

In [82]:
pd.DataFrame(results_dict).T

fit_time         score_time  \
dummy                                 0.002 (+/- 0.002)  0.000 (+/- 0.000)   
imp + scaling + ohe + KNN             0.046 (+/- 0.008)  0.434 (+/- 0.062)   
imp + scaling + ohe + SVR (default)  15.337 (+/- 0.311)  7.757 (+/- 0.112)   
imp + scaling + ohe + SVR (C=10000)  14.216 (+/- 0.499)  7.772 (+/- 0.208)   

                                             test_score         train_score  
dummy                                -0.001 (+/- 0.001)   0.000 (+/- 0.000)  
imp + scaling + ohe + KNN             0.706 (+/- 0.015)   0.806 (+/- 0.015)  
imp + scaling + ohe + SVR (default)  -0.049 (+/- 0.012)  -0.048 (+/- 0.001)  
imp + scaling + ohe + SVR (C=10000)   0.708 (+/- 0.011)   0.713 (+/- 0.017)

With a bigger value for `C` the results are much better. We need to carry out systematic hyperparameter optimization to get better results. (Coming up next week.)

- Note that categorical features are different than free text features. Sometimes there are columns containing free text information and we we'll look at ways to deal with them in the later part of this lecture. 

------------------------

<br><br><br><br><br><br><br><br>
# [Responsible AI] Do we actually want to use certain features for prediction?

- Do you want to use certain features such as **gender** or **race** in prediction?
- Remember that the systems you build are going to be used in some applications. 
- It's extremely important to be mindful of the consequences of including certain features in your predictive model. 
<br><br><br><br><br><br><br><br>

As responsible researchers, we should exclude certain features from the data **even if they improve model performance**.
<br><br><br><br><br>

## OHE with many categories

- Do we have enough data for **rare categories** to learn anything meaningful? 
- How about **grouping** them into bigger categories?
    - Example: country names into continents such as "South America" or "Asia"
- Or having **"other"** category for rare cases? 

<br><br><br><br><br><br><br>
Any decision we make in ML is a hyper-parameter
<br><br><br><br><br><br><br>

### How about the `target` (label) values? Should we preprocess them?

- Generally **no** need for this when doing classification; but, in some cases, **yeah, it could happen**
  - Example: In regression it makes sense in some cases. (More on this later)
- For classification, you often don't need to do much (you might need to apply OrdinalEncoding in some libraries)
  - Example: `sklearn` is fine with categorical labels ($y$-values) for classification problems. 

<br><br><br><br>

# Encoding text data  

In [83]:
toy_spam = [
    [
        "URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!",
        "spam",
    ],
    ["Lol you are always so convincing.", "non spam"],
    ["Nah I don't think he goes to usf, he lives around here though", "non spam"],
    [
        "URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!",
        "spam",
    ],
    [
        "Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030",
        "spam",
    ],
    ["Congrats! I can't wait to see you!!", "non spam"],
]
toy_df = pd.DataFrame(toy_spam, columns=["sms", "target"])
toy_df

sms  \
0                                                                URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!   
1                                                                                                                           Lol you are always so convincing.   
2                                                                                               Nah I don't think he goes to usf, he lives around here though   
3                                                                                 URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!   
4  Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030   
5                                                                                                                         Congrats! I can't wait to see you!!   

     target  
0      spam  
1  non spam  
2  non spam  
3      spam  
4      spam  
5  non spam

## Spam/non spam toy example 

- What if the feature is in the form of raw text?
- The feature `sms` below is neither categorical nor ordinal. 
- How can we encode it so that we can pass it to the machine learning algorithms we have seen so far? 

In [84]:
toy_df

sms  \
0                                                                URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!   
1                                                                                                                           Lol you are always so convincing.   
2                                                                                               Nah I don't think he goes to usf, he lives around here though   
3                                                                                 URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!   
4  Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030   
5                                                                                                                         Congrats! I can't wait to see you!!   

     target  
0      spam  
1  non spam  
2  non spam  
3      spam  
4      spam  
5  non spam

- How can we encode or represent raw text data into fixed number of features so that we can learn some useful patterns from it?  
- This is a well studied problem in the field of **Natural Language Processing (NLP)**, which is concerned with giving computers the ability to understand written and spoken language. 
- Some popular representations of raw text include: 
    - **Bag of words** 
    - TF-IDF
    - Embedding representations 

## Bag of words (BOW) representation (unigram model)

- One of the most popular representation of raw text 
- Ignores the syntax and word order
- It has two components: 
    - The vocabulary (all unique words in all documents) 
    - A value indicating either the presence or absence or the count of each word in the document. 


<center>
<img src='../img/bag-of-words.png' width="600">
</center>

[Source](https://web.stanford.edu/~jurafsky/slp3/4.pdf)       

### Extracting BOW features using `scikit-learn`
- `CountVectorizer`
    - Converts a collection of text documents to a matrix of word counts.  
    - Each row represents a "document" (e.g., a text message in our example). 
    - Each column represents a word in the vocabulary (the set of unique words) in the training data. 
    - Each cell represents how often the word occurs in the document.       

<br><br><br><br><br>
In the Natural Language Processing (NLP) community text data  is referred to as a **corpus** (plural: corpora). 
<br><br><br><br><br>

As usual, start with the documentation:
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

# -------- New Code -----------------------
vec = CountVectorizer()
X_counts = vec.fit_transform(toy_df["sms"])
# -----------------------------------------
bow_df = pd.DataFrame(
    X_counts.toarray(), columns=vec.get_feature_names_out(), index=toy_df["sms"]
)
bow_df

08002986030  \
sms                                                                                                                                                                       
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                          0   
Lol you are always so convincing.                                                                                                                                     0   
Nah I don't think he goes to usf, he lives around here though                                                                                                         0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                           0   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030            1   
Congrats! I can't wait to see you!!                                                                                                                                   0   

                                                                                                                                                            100000  \
sms                                                                                                                                                                  
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                     0   
Lol you are always so convincing.                                                                                                                                0   
Nah I don't think he goes to usf, he lives around here though                                                                                                    0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                      1   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030       0   
Congrats! I can't wait to see you!!                                                                                                                              0   

                                                                                                                                                            11  \
sms                                                                                                                                                              
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                 0   
Lol you are always so convincing.                                                                                                                            0   
Nah I don't think he goes to usf, he lives around here though                                                                                                0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                  0   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030   1   
Congrats! I can't wait to see you!!                                                                                                                          0   

                                                                                                                                                            900  

In [86]:
X_counts

<6x61 sparse matrix of type '<class 'numpy.int64'>'
	with 71 stored elements in Compressed Sparse Row format>

<br><br><br><br>
With `CountVectorizer` you need to define separate `CountVectorizer` transformers for each text column, if you have more than one text columns.    

In [87]:
type(toy_df["sms"])

pandas.core.series.Series

### Why sparse matrices? 

- Most words do not appear in a given document.
- We get massive computational savings if we only store the nonzero elements.
- There is a bit of overhead, because we also need to store the locations:
    - e.g. "location (3,27): 1".
    
- However, if the fraction of nonzero is small, this is a huge win.

In [88]:
print("The total number of elements: ", np.prod(X_counts.shape))
print("The number of non-zero elements: ", X_counts.nnz)
print(
    "Proportion of non-zero elements: %0.4f" % (X_counts.nnz / np.prod(X_counts.shape))
)
print(
    "The value at cell 3,%d is: %d"
    % (vec.vocabulary_["jackpot"], X_counts[3, vec.vocabulary_["jackpot"]])
)

The total number of elements:  366
The number of non-zero elements:  71
Proportion of non-zero elements: 0.1940
The value at cell 3,27 is: 1


<br><br><br><br><br><br>
**Reminder/Note:`OneHotEncoder` and sparse features**
- By default, `OneHotEncoder` also creates sparse features. 
- You could set `sparse=False` to get a regular `numpy` array. 
- If there are a huge number of categories, it may be beneficial to keep them sparse.
- For smaller number of categories, it doesn't matter much.
<br><br><br><br><br><br>


### Important hyperparameters of `CountVectorizer` 

Check the doc: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
- `binary`
    - whether to use absence/presence feature values or counts (If True, all non zero counts are set to 1)
- `max_features`
    - only consider top `max_features` ordered by frequency in the corpus
- `max_df`
    - ignore features which occur in more than `max_df` documents 
- `min_df` 
    - ignore features which occur in less than `min_df` documents 
- `ngram_range`
    - consider word sequences in the given range 

Let's look at all features, i.e., words (along with their frequencies).

In [89]:
vec = CountVectorizer()
X_counts = vec.fit_transform(toy_df["sms"])
bow_df = pd.DataFrame(
    X_counts.toarray(), columns=vec.get_feature_names(), index=toy_df["sms"]
)
bow_df

/Users/amirabdi/miniconda3/envs/cpsc330/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


08002986030  \
sms                                                                                                                                                                       
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                          0   
Lol you are always so convincing.                                                                                                                                     0   
Nah I don't think he goes to usf, he lives around here though                                                                                                         0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                           0   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030            1   
Congrats! I can't wait to see you!!                                                                                                                                   0   

                                                                                                                                                            100000  \
sms                                                                                                                                                                  
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                     0   
Lol you are always so convincing.                                                                                                                                0   
Nah I don't think he goes to usf, he lives around here though                                                                                                    0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                      1   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030       0   
Congrats! I can't wait to see you!!                                                                                                                              0   

                                                                                                                                                            11  \
sms                                                                                                                                                              
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                 0   
Lol you are always so convincing.                                                                                                                            0   
Nah I don't think he goes to usf, he lives around here though                                                                                                0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                  0   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030   1   
Congrats! I can't wait to see you!!                                                                                                                          0   

                                                                                                                                                            900  

When we use `max_features=8`, we limit the number of features to 8

In [90]:
vec_binary = CountVectorizer(max_features=8)  # --> change: max_features=8
X_counts = vec_binary.fit_transform(toy_df["sms"])
bow_df = pd.DataFrame(
    X_counts.toarray(), columns=vec_binary.get_feature_names_out(), index=toy_df["sms"]
)
bow_df

free  \
sms                                                                                                                                                                
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                   0   
Lol you are always so convincing.                                                                                                                              0   
Nah I don't think he goes to usf, he lives around here though                                                                                                  0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                    1   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030     2   
Congrats! I can't wait to see you!!                                                                                                                            0   

                                                                                                                                                            have  \
sms                                                                                                                                                                
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                   1   
Lol you are always so convincing.                                                                                                                              0   
Nah I don't think he goes to usf, he lives around here though                                                                                                  0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                    1   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030     0   
Congrats! I can't wait to see you!!                                                                                                                            0   

                                                                                                                                                            mobile  \
sms                                                                                                                                                                  
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                     0   
Lol you are always so convincing.                                                                                                                                0   
Nah I don't think he goes to usf, he lives around here though                                                                                                    0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                      0   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030       2   
Congrats! I can't wait to see you!!                                                                                                                              0   

                                                                                                                                                            the  \
sms                                   

<br><br><br>
Here, we say: we are only interested in whether the word exists in the doc or not (**ignore the count**)

In [91]:
vec8 = CountVectorizer(binary=True, max_features=8)  # --> change: max_features=8, binary=True
X_counts = vec8.fit_transform(toy_df["sms"])
bow_df = pd.DataFrame(
    X_counts.toarray(), columns=vec8.get_feature_names_out(), index=toy_df["sms"]
)
bow_df.head()

free  \
sms                                                                                                                                                                
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                   0   
Lol you are always so convincing.                                                                                                                              0   
Nah I don't think he goes to usf, he lives around here though                                                                                                  0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                    1   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030     1   

                                                                                                                                                            have  \
sms                                                                                                                                                                
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                   1   
Lol you are always so convincing.                                                                                                                              0   
Nah I don't think he goes to usf, he lives around here though                                                                                                  0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                    1   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030     0   

                                                                                                                                                            mobiles  \
sms                                                                                                                                                                   
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                      0   
Lol you are always so convincing.                                                                                                                                 0   
Nah I don't think he goes to usf, he lives around here though                                                                                                     0   
URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!                                                                                       0   
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030        1   

                                                                                                                                                            months  \
sms                                                                                                                                                                  
URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!                                                                     0   
Lol you are always so convincing.                                                                                                                                0   
Nah I don't think he goe

<br><br><br><br><br><br><br>
**Read the following if the difference in feature names of the above two DataFrames is confusing**

------------
Notice that `vec8` and `vec8_binary` have different vocabularies, which is kind of unexpected behaviour and doesn't match the documentation of `scikit-learn`. 

The **binarization** is done **before limiting the features to `max_features`**, and so now we are actually looking at the document counts (**in how many documents the token occurs**) rather than term count.

The ties in counts between different words makes it even more confusing. I don't think it'll have a big impact on the results but this is good to know! Remember that `scikit-learn` developers are also humans who are prone to make mistakes. So it's always a good habit to question whatever tools we use every now and then. 



In [92]:
vec8 = CountVectorizer(max_features=8)
X_counts = vec8.fit_transform(toy_df["sms"])
pd.DataFrame(
    data=X_counts.sum(axis=0).tolist()[0],
    index=vec8.get_feature_names_out(),
    columns=["counts"],
).sort_values("counts", ascending=False)

counts
to           5
you          4
free         3
have         2
mobile       2
the          2
update       2
urgent       2

In [93]:
vec8_binary = CountVectorizer(binary=True, max_features=8)
X_counts = vec8_binary.fit_transform(toy_df["sms"])
pd.DataFrame(
    data=X_counts.sum(axis=0).tolist()[0],
    index=vec8_binary.get_feature_names_out(),
    columns=["counts"],
).sort_values("counts", ascending=False)

counts
to            4
you           4
free          2
have          2
prize         2
urgent        2
mobiles       1
months        1

------------
<br><br><br><br><br><br><br>

<br><br><br><br><br><br><br><br>
#### Question: 
Is it OK for `CountVectorizer` to be fit on the **Test data** to make sure we include all of its "words"? (afterall, we care about the **count**, right?)

<br><br><br><br><br><br><br><br>

### Preprocessing in `CountVectorizer`

- Note that `CountVectorizer` comes with some default arguments, and does some pre-processing on the text by default
    - example: Converting words to lowercase (`lowercase=True`)
    - example: getting rid of punctuation and special characters (`token_pattern ='(?u)\\b\\w\\w+\\b'`)
    - Learn more here: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html


In [94]:
pipe = make_pipeline(CountVectorizer(), SVC())

In [95]:
pipe.fit(toy_df["sms"], toy_df["target"])

Pipeline(steps=[('countvectorizer', CountVectorizer()), ('svc', SVC())])

In [96]:
# This is a toy problem; 6 samples, we don't have train and test sets.

# if we had a test set, we would have predicted the labels of the test set:
# pipe.predict(X_test["sms"])

# and we would have scored our model on the test set with:
# pipe.score(X_test["sms"], y_test)

### Is this a realistic representation of text data? 

- Of course this is not a great representation of language
    - We are throwing out everything we know about language and losing a lot of information. 
    - **Bag Of Words** assumes that **there is no syntax**, **semantics** and **compositional meaning** in language.  
- But it works surprisingly well for many tasks. 
- We will learn more expressive representations in the coming weeks. 

<br><br>

<br><br><br><br><br><br><br><br>
**[Run this section at home; here, we only focus on the Vocabulary section]** 
----------------------
## Demo of incorporating text features

Recall that we had dropped `song_title` feature when we worked with the Spotify dataset. 

Let's try to include it in our pipeline and examine whether we get better results. 

In [97]:
spotify_df = pd.read_csv("../data/spotify.csv", index_col=0)
X_spotify = spotify_df.drop(columns=["target"])
y_spotify = spotify_df["target"]

In [98]:
X_train, X_test, y_train, y_test = train_test_split(
    X_spotify, y_spotify, test_size=0.2, random_state=123
)

In [99]:
X_train.shape

(1613, 15)

In [100]:
X_train

acousticness  danceability  duration_ms  energy  instrumentalness  key  \
1505      0.004770         0.585       214740   0.614          0.000155   10   
813       0.114000         0.665       216728   0.513          0.303000    0   
615       0.030200         0.798       216585   0.481          0.000000    7   
319       0.106000         0.912       194040   0.317          0.000208    6   
320       0.021100         0.697       236456   0.905          0.893000    6   
...            ...           ...          ...     ...               ...  ...   
2012      0.001060         0.584       274404   0.932          0.002690    1   
1346      0.000021         0.535       203500   0.974          0.000149   10   
1406      0.503000         0.410       256333   0.648          0.000000    7   
1389      0.705000         0.894       222307   0.161          0.003300    4   
1534      0.623000         0.470       394920   0.156          0.187000    2   

      liveness  loudness  mode  speechiness    tempo  time_signature  valence  \
1505    0.0762    -5.594     0       0.0370  114.059             4.0   0.2730   
813     0.1220    -7.314     1       0.3310  100.344             3.0   0.0373   
615     0.1280   -10.488     1       0.3140  127.136             4.0   0.6400   
319     0.0723   -12.719     0       0.0378   99.346             4.0   0.9490   
320     0.1190    -7.787     0       0.0339  119.977             4.0   0.3110   
...        ...       ...   ...          ...      ...             ...      ...   
2012    0.1290    -3.501     1       0.3330   74.976             4.0   0.2110   
1346    0.2630    -3.566     0       0.1720  116.956             4.0   0.4310   
1406    0.2190    -4.469     1       0.0362   60.391             4.0   0.3420   
1389    0.3120   -14.311     1       0.0880  104.968             4.0   0.8180   
1534    0.1040   -17.036     1       0.0399  118.176             4.0   0.0591   

                                                                        song_title  \
1505                                                           Cool for the Summer   
813   Damn Son Where'd You Find This? (feat. Kelly Holiday) - Markus Maximus Remix   
615                                                                      Trill Hoe   
319                                             Who Is He (And What Is He to You?)   
320                                                                         Acamar   
...                                                                            ...   
2012                                           Like A Bitch - Kill The Noise Remix   
1346                                                             Flag of the Beast   
1406                                                          Don't You Cry For Me   
1389                                                  장가갈 수 있을까 Can I Get Married?   
1534                                                                   Blue Ballad   

              artist  
1505     Demi Lovato  
813   Markus Maximus  
615     Western Tink  
319     Bill Withers  
320          Frankey  
...              ...  
2012  Kill The Noise  
1346          Emmure  
1406            Cobi  
1389       Coffeeboy  
1534      Phil Woods  

[1613 rows x 15 columns]

Let's look at the distribution of values in the `song_title` column. 

In [101]:
X_train["song_title"].value_counts()

Pyramids                                     2
Look At Wrist                                2
Baby                                         2
The One                                      2
Best Friend                                  2
                                            ..
City Of Dreams - Radio Edit                  1
Face It                                      1
The Winner Is - from Little Miss Sunshine    1
History                                      1
Blue Ballad                                  1
Name: song_title, Length: 1579, dtype: int64

- Most of the song titles are unique, which makes sense. 
- What would happen if we apply one-hot encoding to this feature? 
- Can we encode this as a text feature? 

In [102]:
X_train.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'song_title',
       'artist'],
      dtype='object')

In [103]:
numeric_features = [
    "acousticness",
    "danceability",
    "duration_ms",
    "energy",
    "instrumentalness",
    "key",
    "liveness",
    "loudness",
    "mode",
    "speechiness",
    "tempo",
    "time_signature",
    "valence",
]
drop_features = ['artist']

# Note that unlike other feature types we are defining `text_feature` as a string and not as a list.
text_feature = "song_title"  # note that we are not creating a list here.

preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (CountVectorizer(max_features=2000, stop_words="english"), text_feature),
    ("drop", drop_features)
)

### Explore the transformed data 

In [104]:
transformed = preprocessor.fit_transform(X_train, y_train)
transformed.shape

(1613, 1897)

In [105]:
vocab = preprocessor.named_transformers_["countvectorizer"].get_feature_names_out()

In [106]:
vocab[40:80]

array(['able', 'abomination', 'absolutely', 'abyss', 'acamar',
       'accelerate', 'ace', 'acoustic', 'adagio', 'adam', 'adios',
       'adorn', 'advent', 'adventure', 'affection', 'afraid', 'afro',
       'agadoo', 'agony', 'agorinha', 'agye', 'ahora', 'ain', 'akon',
       'akue', 'akyiri', 'al', 'alan', 'album', 'alessia', 'alguien',
       'alive', 'allegretto', 'allen', 'aloha', 'aloud', 'alp', 'alright',
       'alternative', 'amanecer'], dtype=object)

In [107]:
vocab.shape

(1884,)

In [108]:
column_names = numeric_features + vocab.tolist()

In [109]:
df = pd.DataFrame(transformed.toarray(), columns=column_names, index=X_train.index)
df

acousticness  danceability  duration_ms    energy  instrumentalness  \
1505     -0.697633     -0.194548    -0.398940 -0.318116         -0.492359   
813      -0.276291      0.295726    -0.374443 -0.795552          0.598355   
615      -0.599540      1.110806    -0.376205 -0.946819         -0.492917   
319      -0.307150      1.809445    -0.654016 -1.722063         -0.492168   
320      -0.634642      0.491835    -0.131344  1.057468          2.723273   
...            ...           ...          ...       ...               ...   
2012     -0.711944     -0.200676     0.336272  1.185100         -0.483229   
1346     -0.715953     -0.500969    -0.537445  1.383637         -0.492380   
1406      1.224228     -1.267021     0.113591 -0.157395         -0.492917   
1389      2.003419      1.699134    -0.305695 -2.459489         -0.481032   
1534      1.687114     -0.899316     1.821334 -2.483125          0.180574   

           key  liveness  loudness      mode  speechiness  ...  너와의  루시아   변명  \
1505  1.275623 -0.737898  0.395794 -1.280599    -0.617752  ...  0.0  0.0  0.0   
813  -1.487342 -0.438792 -0.052394  0.780884     2.728394  ...  0.0  0.0  0.0   
615   0.446734 -0.399607 -0.879457  0.780884     2.534909  ...  0.0  0.0  0.0   
319   0.170437 -0.763368 -1.460798 -1.280599    -0.608647  ...  0.0  0.0  0.0   
320   0.170437 -0.458384 -0.175645 -1.280599    -0.653035  ...  0.0  0.0  0.0   
...        ...       ...       ...       ...          ...  ...  ...  ...  ...   
2012 -1.211046 -0.393077  0.941176  0.780884     2.751157  ...  0.0  0.0  0.0   
1346  1.275623  0.482038  0.924239 -1.280599     0.918743  ...  0.0  0.0  0.0   
1406  0.446734  0.194687  0.688940  0.780884    -0.626857  ...  0.0  0.0  0.0   
1389 -0.382156  0.802042 -1.875632  0.780884    -0.037298  ...  0.0  0.0  0.0   
1534 -0.934749 -0.556344 -2.585697  0.780884    -0.584746  ...  0.0  0.0  0.0   

       여기  이곳에서  이대로  있어줘요  있을까  장가갈   지금  
1505  0.0   0.0  0.0   0.0  0.0  0.0  0.0  
813   0.0   0.0  0.0   0.0  0.0  0.0  0.0  
615   0.0   0.0  0.0   0.0  0.0  0.0  0.0  
319   0.0   0.0  0.0   0.0  0.0  0.0  0.0  
320   0.0   0.0  0.0   0.0  0.0  0.0  0.0  
...   ...   ...  ...   ...  ...  ...  ...  
2012  0.0   0.0  0.0   0.0  0.0  0.0  0.0  
1346  0.0   0.0  0.0   0.0  0.0  0.0  0.0  
1406  0.0   0.0  0.0   0.0  0.0  0.0  0.0  
1389  0.0   0.0  0.0   0.0  1.0  1.0  0.0  
1534  0.0   0.0  0.0   0.0  0.0  0.0  0.0  

[1613 rows x 1897 columns]

### Explore the learned vocabulary 

In [110]:
vocab[0:10]

array(['000', '10', '100', '10cm', '11', '112', '12', '1208', '144', '18'],
      dtype=object)

In [111]:
vocab[500:510]

array(['duele', 'duet', 'duke', 'dustland', 'dutchie', 'dynamite',
       'earth', 'easy', 'eazy', 'echelon'], dtype=object)

In [112]:
vocab[1800:1810]

array(['wide', 'wifey', 'wild', 'wildcard', 'wildfire', 'wiley',
       'willing', 'win', 'wind', 'window'], dtype=object)

In [113]:
vocab[0::100]

array(['000', 'ap', 'blind', 'cha', 'dallask', 'duele', 'flashlight',
       'grace', 'icarus', 'lafa', 'making', 'neck', 'pharaohs', 'redeem',
       'seeb', 'soundtrack', 'talons', 'unanswered', 'wide'], dtype=object)

<br><br><br><br><br><br>
**[Explore on your own]**
-------------

Let's find songs containing the word _earth_ in them. 

In [114]:
earth_index_vocab = np.where(vocab == "earth")
print(earth_index_vocab)
print('index of "earth" in the vocabulary list is:', earth_index_vocab[0][0])

(array([506]),)
index of "earth" in the vocabulary list is: 506


In [115]:
earth_index_in_df = len(numeric_features) + earth_index_vocab[0][0]
earth_index_in_df

519

In [116]:
earth_songs = df[df.iloc[:, earth_index_in_df] == 1]
earth_songs.iloc[:, earth_index_in_df - 2 : earth_index_in_df + 2]

dutchie  dynamite  earth  easy
1851      0.0       0.0    1.0   0.0
1948      0.0       0.0    1.0   0.0

In [117]:
earth_songs.index

Int64Index([1851, 1948], dtype='int64')

In [118]:
X_train.loc[earth_songs.index]["song_title"]

1851             Softest Place On Earth
1948    Earth Song - Remastered Version
Name: song_title, dtype: object

------------
<br><br><br><br><br><br><br>

### Model building 

Let's create a pipeline using SVC. 
- SVC works well with sparse features. 

In [119]:
pipe = make_pipeline(preprocessor, SVC())

In [120]:
results = pd.DataFrame(cross_validate(pipe, X_train, y_train, return_train_score=True))
print('validation score:', results.mean()['test_score'])

validation score: 0.7377516681729899


--------
- Is our CV **improving** after incorporating this feature?
- Let's examine what numbers we get when we don't include it. 
--------

In [121]:
pipe_num = make_pipeline(StandardScaler(), SVC())

X_train_num = X_train.drop(columns=["song_title", 'artist'])

In [122]:
results = pd.DataFrame(
    cross_validate(pipe_num, X_train_num, y_train, return_train_score=True)
)
print('validation score:', results.mean()['test_score'])

validation score: 0.7389977501298002


- Not a big difference in the results. 

- What about the `artist` column?
- Does it make sense to apply BOW encoding to it? 
- Let's look at the distribution of values in the `artist` column. 

In [123]:
X_train['artist'].value_counts()

Drake              14
Disclosure         12
Rick Ross          11
WALK THE MOON      10
Crystal Castles     8
                   ..
Classixx            1
Jordan Feliz        1
Travis Hayes        1
The Silvertones     1
Phil Woods          1
Name: artist, Length: 1131, dtype: int64

In [124]:
most_frequent = X_train["artist"].value_counts().iloc[:15]
most_frequent

Drake              14
Disclosure         12
Rick Ross          11
WALK THE MOON      10
Crystal Castles     8
Big Time Rush       8
FIDLAR              8
Fall Out Boy        8
Demi Lovato         7
Kanye West          7
Kina Grannis        7
Backstreet Boys     7
Beach House         6
Young Thug          6
*NSYNC              6
Name: artist, dtype: int64

- We have many unique artists. Probably it's not worth to create a "other" category here. 

In [125]:
numeric_features = [
    "acousticness",
    "danceability",
    "duration_ms",
    "energy",
    "instrumentalness",
    "key",
    "liveness",
    "loudness",
    "mode",
    "speechiness",
    "tempo",
    "time_signature",
    "valence",
]
categorical_features = ['artist']
text_feature = "song_title"  # note that we are not creating a list here.

preprocessor_artist = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(sparse=False, dtype=int, handle_unknown="ignore", categories=[most_frequent.index.values]), categorical_features),
    (CountVectorizer(max_features=2000, stop_words="english"), text_feature),
)

In [126]:
pipe = make_pipeline(preprocessor_artist, SVC())

In [127]:
results = pd.DataFrame(cross_validate(pipe, X_train, y_train, return_train_score=True))
print('validation score:', results.mean()['test_score'])

validation score: 0.7408514893371537


**Tiny bit** improvement in the mean CV scores but we are still overfitting. 

<br><br><br><br><br><br><br>
When adding a feature doesn't add much value to the product, but, adds complexity, we sometimes decide not to include the feature.

We call such features with **minimal impact** the **epsilon features**
<br><br><br><br><br><br><br>

<br><br>

## ❓❓ Questions for you 

### (iClicker) Exercise 6.2 

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

- (A) `handle_unknown="ignore"` would treat all unknown categories equally. 
- (B) As you increase the value for `max_features` hyperparameter of `CountVectorizer` the training score is likely to go up. 
- (C) Suppose you are encoding text data using `CountVectorizer`. If you encounter a word in the validation or the test split that's not available in the training data, we'll get an error. 
- (D) In the code below, inside `cross_validate`, each fold might have slightly different number of features (columns) in the fold.

```
pipe = (CountVectorizer(), SVC())
cross_validate(pipe, X_train, y_train)
```

<br><br><br><br><br>
## Other Language Preprocessors and Models

### TF-IDF (term frequency–inverse document frequency)

In [128]:
toy_spam = [
    [
        "URGENT!! As a valued network customer you have been selected to receive a £900 prize reward!",
        "spam",
    ],
    ["Lol you are always so convincing.", "non spam"],
    ["Nah I don't think he goes to usf, he lives around here though", "non spam"],
    [
        "URGENT! You have won a 1 week FREE membership in our £100000 prize Jackpot!",
        "spam",
    ],
    [
        "Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030",
        "spam",
    ],
    ["Congrats! I can't wait to see you!!", "non spam"],
]
toy_df = pd.DataFrame(toy_spam, columns=["sms", "target"])

In [129]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = toy_df['sms']

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [130]:
print(vectorizer.get_feature_names_out())

['08002986030' '100000' '11' '900' 'always' 'are' 'around' 'as' 'been'
 'call' 'camera' 'can' 'co' 'colour' 'congrats' 'convincing' 'customer'
 'don' 'entitled' 'for' 'free' 'goes' 'had' 'have' 'he' 'here' 'in'
 'jackpot' 'latest' 'lives' 'lol' 'membership' 'mobile' 'mobiles' 'months'
 'more' 'nah' 'network' 'on' 'or' 'our' 'prize' 'receive' 'reward' 'see'
 'selected' 'so' 'the' 'think' 'though' 'to' 'update' 'urgent' 'usf'
 'valued' 'wait' 'week' 'with' 'won' 'you' 'your']


In [131]:
X.toarray()

array([[0.        , 0.        , 0.        , 0.29208838, 0.        ,
        0.        , 0.        , 0.29208838, 0.29208838, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.29208838, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.23951665, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.29208838, 0.        , 0.        ,
        0.        , 0.23951665, 0.29208838, 0.29208838, 0.        ,
        0.29208838, 0.        , 0.        , 0.        , 0.        ,
        0.1732841 , 0.        , 0.23951665, 0.        , 0.29208838,
        0.        , 0.        , 0.        , 0.        , 0.1732841 ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.43225867,
        0.43225867, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.     

<br><br><br><br>

<br><br><br><br><br><br>
## What is n-gram?

That's something you folks will learn about because we didn't have time to cover :D
<br><br><br><br><br><br>

## What did we learn today?

- Motivation to use `ColumnTransformer`
- `ColumnTransformer` syntax
- Defining transformers with multiple transformations
- How to visualize transformed features in a dataframe 
- More on ordinal features 
- Different arguments `OneHotEncoder`
    - `handle_unknow="ignore"`
    - `if_binary`
- Dealing with text features
    - Bag of words representation: `CountVectorizer`

![](../img/eva-talksoon.png)